In [ ]:
import lightning as L
from importlib import reload

import sys
from pyprojroot import here
sys.path.append(str(here()))
from models import model_definitions as MD

In [ ]:
reload(MD)
distil_3 = MD.DistilBertFinetune.load_from_checkpoint(here("models/model_checkpoints/distil_bert_finetuned_3.ckpt"))